# Get the data

Depending on each case your data may look a little bit different but we should start by obtaining asset data for our next steps. We will use Binance API to obtain information about a set of assets and derive our calculations from there.

In [1]:
import pandas as pd

data = pd.read_csv("../../data/binance_data.csv")
data

,Unnamed: 0,Asset,Open time,Open,High,Low,Close,Volume,Closing time,Quote asset vol,Num traders,Taker buy base asset vol,Taker buy quote asset vol,To be ignored
0,0,ETHBTC,1633392000000,0.068792,0.069164,0.067950,0.068292,93721.2228,1633478399999,6424.682503,193492,50491.3349,3460.997314,0
1,1,ETHBTC,1633478400000,0.068291,0.068808,0.064412,0.064618,139166.2235,1633564799999,9212.504998,295223,69105.8866,4574.803694,0
2,2,ETHBTC,1633564800000,0.064621,0.067230,0.064000,0.066703,104537.8558,1633651199999,6871.880449,229116,52606.9135,3459.551523,0
3,3,ETHBTC,1633651200000,0.066698,0.066999,0.065000,0.066043,83324.2882,1633737599999,5505.205826,195670,40485.2455,2675.035626,0
4,4,ETHBTC,1633737600000,0.066042,0.066139,0.064976,0.065050,54157.6603,1633823999999,3551.371032,132380,26596.6514,1743.969841,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,361,QTUMETH,1664582400000,0.002152,0.002171,0.002141,0.002171,1124.1000,1664668799999,2.430742,58,753.3000,1.630076,0
1826,362,QTUMETH,1664668800000,0.002168,0.002185,0.002145,0.002168,4146.0000,1664755199999,8.995018,85,2048.6000,4.470011,0
1827,363,QTUMETH,1664755200000,0.002166,0.002185,0.002152,0.002166,1052.8000,1664841599999,2.282456,44,733.0000,1.591202,0
1828,364,QTUMETH,1664841600000,0.002153,0.002163,0.002127,0.002127,204.6000,1664927999999,0.438906,17,46.4000,0.100051,0


Following this we will calculate the average expected revenue for each asset. This is done by getting the difference between *opening* and *closing* time scaled to the opening price. This way we make sure each asset is independently evaluated.

We will also compute the covariance between each asset so that we can consider this values as part of our portfolio diversification constraint.

In [3]:
from cmath import exp
import numpy as np

# Unique asset list
asset_list = data["Asset"].unique()

stddev = []
for asset in asset_list:
    close_price = np.array(data[data["Asset"] == asset]["Close"].astype("float"))
        
    # Sign will be used to indicate the value gradient direction
    sigma = close_price.std()
    stddev.append(sigma)

It is important to know what the closing price for future valuation.

In [4]:
filter = data.groupby("Asset").agg({"Open time":max}).reset_index()
price = data.merge(filter, how='inner').drop_duplicates()
price["Stddev"] = stddev
dataframe = price[["Asset","Close", "Stddev"]].to_dict('records')
dataframe

[{'Asset': 'ETHBTC', 'Close': 0.067077, 'Stddev': 0.00754239492210189},
 {'Asset': 'LTCBTC', 'Close': 0.00271, 'Stddev': 0.00040520207128267476},
 {'Asset': 'BNBBTC', 'Close': 0.014593, 'Stddev': 0.0017180858591516383},
 {'Asset': 'NEOBTC', 'Close': 0.000433, 'Stddev': 0.0001107564237333007},
 {'Asset': 'QTUMETH', 'Close': 0.002126, 'Stddev': 0.0004893312804949519}]

We will store this information so that it can be used later.

In [5]:
import json

# Serializing json  
json_object = json.dumps(dataframe, indent = 4)

with open("binance-data.json", "w") as file:
    file.write(json_object)